# 损失函数与反向传播 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- 损失函数与反向传播



## 1.损失函数（Loss Function）
Loss Function可以被用于衡量模型的预期输出（output）和目标值（target）之间的误差。Loss Function的计算方式有很多，其中L1Loss采用的是MAE（Mean Absolute Error）计算方式：
$$l_n = \lvert x_n - y_n \rvert$$
$$L = \{ l_1, l_2, ..., l_N\}$$
再根据`reduction`参数决定对L求和或求均值

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html

In [7]:
import torch

inputs = torch.tensor([1, 2, 3], dtype=torch.float)
targets = torch.tensor([1,2, 5], dtype=torch.float)

#reshape的目的是添加batch_size这一数据，实际上这个例子中不需要reshape
inputs = torch.reshape(inputs, (1, 1, 1, 3))
targets = torch.reshape(targets, (1, 1, 1, 3))

loss = torch.nn.L1Loss(reduction="mean")#MAE
'''
实例化loss function。常用参数：
reduction -> 若为"mean"对L求平均值（MAE），若为"sum"，对L求和
'''
result = loss(inputs, targets)
result

tensor(0.6667)

另一种常用的计算方式是MSE（mean squared error）：
$$l_n = (x_n - y_n)^2$$
$$L = \{ l_1, l_2, ..., l_N\}$$
其余部分与MAE相同

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html

In [10]:
loss_mse = torch.nn.MSELoss(reduction="mean")#MAE
'''
实例化loss function。常用参数：
reduction -> 若为"mean"对L求平均值（MAE），若为"sum"，对L求和
'''
result_mse = loss_mse(inputs, targets)
result_mse

tensor(1.3333)

在训练分类问题时，也常用到交叉熵（Cross-Entropy Loss）计算

关于交叉熵的数学计算，参考：https://www.bilibili.com/video/BV1MP411r7sL

官方文档：https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

In [25]:
x = torch.tensor([0.1, 0.2, 0.3])
y = torch.tensor([1])
x = torch.reshape(x, (1, 3))
loss_cross = torch.nn.CrossEntropyLoss()
result_cross = loss_cross(x, y)
'''
loss_cross的输入x是对于所有类别预测的结果，y是target，即该数据所真实对应的类别
'''
result_cross

tensor(1.1019)

## 2.反向传播函数
Loss Function为更新输出提供依据。即通过Loss Function的返回值可以确定反向传播函数。可以运用反向传播函数求神经网络中神经元的梯度

In [26]:
import torchvision

dataset = torchvision.datasets.CIFAR10(root = "../dataset", train=False, download=False, transform=torchvision.transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1, drop_last=True)

In [28]:
class Tudui2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(kernel_size=5, padding=2, in_channels=32, out_channels=64),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=1024, out_features=10)
        )
    def forward(self, x):
        x = self.model1(x)
        return x
tudui = Tudui2()

In [ ]:
for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    print(output)
    print(targets)

    result_loss = loss_cross(output, targets)
    print(result_loss)

    result_loss.backward()
    #运行backwards方法，会在神经网络中计算每个神经元权重的梯度（gradient)并添加到神经网络的属性中

## 3.优化器（optimizer）
优化器可以利用梯度，对神经网络进行操作，起到降低误差的目的。优化器的类在`torch.optim`中，创建优化器需要对其进行实例化。

官方文档：https://pytorch.org/docs/stable/optim.html

pytorch提供的优化器种类有很多，这里以SGD（Stochastic Gradient Descent，随机梯度下降）优化器举例。


In [ ]:
tudui = Tudui2()

optim = torch.optim.SGD(params=tudui.parameters(), lr=0.001) 
'''
创建优化器。常用参数：
params -> 形参，调用神经网络方法即可获得
lr -> 学习率，一般先设置的较搞进行学习，再进行调整
'''
for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = tudui(imgs)
        #初始化梯度为0
        optim.zero_grad()
        #获取loss值
        result_loss = loss_cross(outputs, targets)#loss function不一定是单一的batch
        #用loss值确定的反向传播函数获取梯度
        result_loss.backward()
        #用优化器进行参数优化
        optim.step()
        running_loss += result_loss
    print(running_loss)
